In [211]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

In [212]:
results_df = pd.read_csv("Datasets/results.csv")

In [213]:
results_df.head()

,date,team_1,team_2,_map,result_1,result_2,map_winner,starting_ct,ct_1,t_2,t_1,ct_2,event_id,match_id,rank_1,rank_2,map_wins_1,map_wins_2,match_winner
0,2020-03-18,Recon 5,TeamOne,Dust2,0,16,2,2,0,1,0,15,5151,2340454,62,63,0,2,2
1,2020-03-18,Recon 5,TeamOne,Inferno,13,16,2,2,8,6,5,10,5151,2340454,62,63,0,2,2
2,2020-03-18,New England Whalers,Station7,Inferno,12,16,2,1,9,6,3,10,5243,2340461,140,118,12,16,2
3,2020-03-18,Rugratz,Bad News Bears,Inferno,7,16,2,2,0,8,7,8,5151,2340453,61,38,0,2,2
4,2020-03-18,Rugratz,Bad News Bears,Vertigo,8,16,2,2,4,5,4,11,5151,2340453,61,38,0,2,2


In [214]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45773 entries, 0 to 45772
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          45773 non-null  object
 1   team_1        45773 non-null  object
 2   team_2        45773 non-null  object
 3   _map          45773 non-null  object
 4   result_1      45773 non-null  int64 
 5   result_2      45773 non-null  int64 
 6   map_winner    45773 non-null  int64 
 7   starting_ct   45773 non-null  int64 
 8   ct_1          45773 non-null  int64 
 9   t_2           45773 non-null  int64 
 10  t_1           45773 non-null  int64 
 11  ct_2          45773 non-null  int64 
 12  event_id      45773 non-null  int64 
 13  match_id      45773 non-null  int64 
 14  rank_1        45773 non-null  int64 
 15  rank_2        45773 non-null  int64 
 16  map_wins_1    45773 non-null  int64 
 17  map_wins_2    45773 non-null  int64 
 18  match_winner  45773 non-null  int64 
dtypes: i

In [215]:
results_df.describe()

,result_1,result_2,map_winner,starting_ct,ct_1,t_2,t_1,ct_2,event_id,match_id,rank_1,rank_2,map_wins_1,map_wins_2,match_winner
count,45773.000000,45773.000000,45773.000000,45773.000000,45773.000000,45773.000000,45773.000000,45773.000000,45773.000000,4.577300e+04,45773.000000,45773.000000,45773.000000,45773.000000,45773.000000
mean,13.385489,12.731545,1.462478,1.501387,6.659515,6.057283,6.375571,6.335482,3580.790859,2.321278e+06,68.394228,77.289428,4.896708,4.597710,1.464204
std,4.417713,4.715722,0.498596,0.500004,3.257815,3.161310,3.184036,3.285287,904.337168,1.170610e+04,61.835919,65.708309,6.044017,5.931323,0.498722
min,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,820.000000,2.299001e+06,1.000000,1.000000,0.000000,0.000000,1.000000
25%,11.000000,9.000000,1.000000,1.000000,4.000000,4.000000,4.000000,4.000000,2818.000000,2.311132e+06,22.000000,28.000000,1.000000,1.000000,1.000000
50%,16.000000,16.000000,1.000000,2.000000,7.000000,6.000000,6.000000,6.000000,3572.000000,2.321956e+06,49.000000,58.000000,2.000000,2.000000,1.000000
75%,16.000000,16.000000,2.000000,2.000000,9.000000,8.000000,9.000000,9.000000,4346.000000,2.331691e+06,96.000000,110.000000,8.000000,7.000000,2.000000
max,38.000000,41.000000,2.000000,2.000000,15.000000,16.000000,15.000000,15.000000,5248.000000,2.340461e+06,404.000000,404.000000,34.000000,34.000000,2.000000


In [216]:
results_df.isnull().sum()

date            0
team_1          0
team_2          0
_map            0
result_1        0
result_2        0
map_winner      0
starting_ct     0
ct_1            0
t_2             0
t_1             0
ct_2            0
event_id        0
match_id        0
rank_1          0
rank_2          0
map_wins_1      0
map_wins_2      0
match_winner    0
dtype: int64

In [217]:
results_df['teams'] = results_df['team_1'] +","+ results_df['team_2']
results_df

,date,team_1,team_2,_map,result_1,result_2,map_winner,starting_ct,ct_1,t_2,t_1,ct_2,event_id,match_id,rank_1,rank_2,map_wins_1,map_wins_2,match_winner,teams
0,2020-03-18,Recon 5,TeamOne,Dust2,0,16,2,2,0,1,0,15,5151,2340454,62,63,0,2,2,"Recon 5,TeamOne"
1,2020-03-18,Recon 5,TeamOne,Inferno,13,16,2,2,8,6,5,10,5151,2340454,62,63,0,2,2,"Recon 5,TeamOne"
2,2020-03-18,New England Whalers,Station7,Inferno,12,16,2,1,9,6,3,10,5243,2340461,140,118,12,16,2,"New England Whalers,Station7"
3,2020-03-18,Rugratz,Bad News Bears,Inferno,7,16,2,2,0,8,7,8,5151,2340453,61,38,0,2,2,"Rugratz,Bad News Bears"
4,2020-03-18,Rugratz,Bad News Bears,Vertigo,8,16,2,2,4,5,4,11,5151,2340453,61,38,0,2,2,"Rugratz,Bad News Bears"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45768,2015-11-05,G2,E-frag.net,Inferno,13,16,2,1,8,7,5,9,1970,2299059,7,16,1,2,2,"G2,E-frag.net"
45769,2015-11-05,G2,E-frag.net,Dust2,16,13,1,1,10,5,6,8,1970,2299059,7,16,1,2,2,"G2,E-frag.net"
45770,2015-11-04,CLG,Liquid,Inferno,16,12,1,1,7,8,9,4,1934,2299011,10,14,16,12,1,"CLG,Liquid"
45771,2015-11-03,NiP,Dignitas,Train,16,4,1,2,4,1,12,3,1934,2299001,6,12,16,4,1,"NiP,Dignitas"


In [218]:
results_df, test_df = train_test_split(results_df, test_size=0.20)
train_df, dev_df = train_test_split(results_df, test_size=0.20)

In [219]:
tfidf = TfidfVectorizer(max_features=1000)

x_train = tfidf.fit_transform(train_df['teams']).toarray()
y_train = np.asarray(train_df['match_winner'])

In [220]:
x_dev = tfidf.transform(dev_df['teams']).toarray()
y_dev = np.asarray(dev_df['match_winner'])

In [221]:
x_test = tfidf.fit_transform(test_df['teams']).toarray()
y_test = np.asarray(test_df['match_winner'])

In [222]:
print(x_train.shape)
print(y_train.shape)
print(x_dev.shape)
print(y_dev.shape)
print(x_test.shape)
print(y_test.shape)

(29294, 1000)
(29294,)
(7324, 1000)
(7324,)
(9155, 1000)
(9155,)


In [223]:
train_df['teams_transform'] = tfidf.fit_transform(train_df['teams']).toarray()
train_df['map_transform'] = tfidf.fit_transform(train_df['_map']).toarray()

ValueError: Wrong number of items passed 1000, placement implies 1

In [ ]:
Features = ["teams_transform", "map_transform", "rank_1", "rank_2"]
x = train_df[Features]
y = train_df["match_winner"]

In [ ]:
models = [LinearSVC(),KNeighborsClassifier(),RandomForestClassifier(),DecisionTreeClassifier(),GradientBoostingClassifier(),GaussianNB()]

model_names=['LinearSVM','KNearestNeighbors','RandomForestClassifier','DecisionTree','GradientBoostingClassifier','GaussianNB']

acc = []

for model in range(len(models)):
    classifiers = models[model]
    classifiers.fit(x_train,y_train)
    pred = classifiers.predict(x_test)
    acc.append(accuracy_score(pred,y_test))

d={'Modelling Algo':model_names,'Accuracy':acc}
d

In [ ]:
acc_frame=pd.DataFrame(d)
acc_frame.sort_values(by = 'Accuracy', ascending = False)

In [ ]:
gradient_boost = GradientBoostingClassifier()

In [ ]:
gradient_boost.fit(x,y)
